In [1]:
import pandas as pd
import glob

In [4]:
# Merging csv files (without repeating header)
interesting_files = glob.glob("*.csv") 
df = pd.concat((pd.read_csv(f, header = 0) for f in interesting_files))
df

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,Author,Body,Date,Subreddit,Title,URL,Unnamed: 0,tfidf,topic,topic 0,topic 1,topic 2,topic 2,topic 3,topic 4,topic 5,words
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.03363,NaN,19
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.03491,NaN,NaN,NaN,NaN,NaN,NaN,able
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.024724,NaN,NaN,NaN,advice
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.043861,NaN,NaN,ago
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.049829,NaN,NaN,NaN,NaN,NaN,ask
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7149,Prestoric_user,I want to sell my ps4 since i dont use it anym...,1.585612e+09,Anxiety,I want to sell my console but its hard,/r/Anxiety/comments/fs251x/i_want_to_sell_my_c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7150,Gametest014,So yesterday(this morning) I went to sleep at ...,1.585612e+09,Anxiety,Just looking for advice,/r/Anxiety/comments/fs260r/just_looking_for_ad...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7151,throw-away-3005,I took the first bath in our new house and it ...,1.585612e+09,Anxiety,My first bath,/r/Anxiety/comments/fs27t3/my_first_bath/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7152,AnxiousFishermen,Without meaning too i always find my self cros...,1.585613e+09,Anxiety,Anyone else always crossing their legs?,/r/Anxiety/comments/fs28s1/anyone_else_always_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
